# Nepal Earthquake Damage Prediction

This notebook predicts the severity of building damage caused by the Nepal earthquake using a Random Forest Classifier. The workflow includes data preprocessing, feature engineering, model training, hyperparameter tuning, and evaluation.


## 1. Import Libraries
Import all necessary libraries for data manipulation, visualization, and modeling.


In [1]:
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation
import matplotlib.pyplot as plt  # For visualization
from category_encoders import OneHotEncoder, OrdinalEncoder  # For encoding categorical variables
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils.validation import check_is_fitted
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier


## 2. Load and Preview Data
Load the dataset and display the first few rows to understand its structure.


In [3]:
df = pd.read_csv('building_structure.csv')
df.head()


,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,...,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,damage_grade,technical_solution_proposed
0,120101000011,12,1207,120703,1,1,9,288,9,9,...,0,0,0,1,0,0,0,Damaged-Used in risk,Grade 3,Major repair
1,120101000021,12,1207,120703,1,1,15,364,9,9,...,0,0,0,1,0,0,0,Damaged-Repaired and used,Grade 5,Reconstruction
2,120101000031,12,1207,120703,1,1,20,384,9,9,...,0,0,0,0,0,0,0,Damaged-Repaired and used,Grade 2,Minor repair
3,120101000041,12,1207,120703,1,1,20,312,9,9,...,0,0,0,0,0,0,0,Damaged-Repaired and used,Grade 2,Minor repair
4,120101000051,12,1207,120703,1,1,30,308,9,9,...,0,0,0,0,0,0,0,Damaged-Repaired and used,Grade 1,Minor repair


## 3. Data Wrangling
Define a function to clean and preprocess the data by removing leaky, high-cardinality, and multicollinear features.


In [4]:
def wrangle(data_path):
    df = pd.read_csv(data_path)
    # Identify leaky features
    drop_col = [col for col in df.columns if 'post_eq' in col]
    drop_col.append('technical_solution_proposed')
    # Remove missing values
    df.dropna(inplace=True)
    # Create binary target
    df['severe_damage'] = df['damage_grade'].str[-1].astype('int')
    df['severe_damage'] = (df['severe_damage'] > 3).astype('int')
    drop_col.append('damage_grade')
    # Drop high cardinality and multicollinear features
    drop_col.append('building_id')
    drop_col.extend(['count_floors_pre_eq', 'ward_id', 'vdcmun_id'])
    df.drop(columns=drop_col, inplace=True)
    return df


## 4. Data Preparation
Apply the wrangling function, subset the data, and separate features and target.


In [5]:
pd.set_option('display.max_columns', None)
df = wrangle('building_structure.csv')
df = df.iloc[:3000, :]  # Use a subset for faster experimentation
print(df.shape)
df.head()


(3000, 23)


,district_id,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,severe_damage
0,12,9,288,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,1,0,0,0,0
1,12,15,364,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,1,0,0,0,1
2,12,20,384,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,0
3,12,20,312,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,0
4,12,30,308,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,0


In [6]:
target = 'severe_damage'
X = df.drop(columns=target)
y = df[target]
print('X shape:', X.shape)
print('y shape:', y.shape)


X shape: (3000, 22)
y shape: (3000,)


## 5. Train-Test Split
Split the data into training and test sets.


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)


X_train shape: (2400, 22)
y_train shape: (2400,)
X_test shape: (600, 22)
y_test shape: (600,)


## 6. Feature Engineering
One-hot encode categorical features using sklearn's ColumnTransformer.


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
categorical_cols = ['land_surface_condition', 'foundation_type', 'roof_type',
                    'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)
X_train_ohe = preprocessor.fit_transform(X_train)
X_test_ohe = preprocessor.transform(X_test)


## 7. Model Training and Hyperparameter Tuning
Train a Random Forest Classifier and use RandomizedSearchCV for hyperparameter tuning.


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 0.5],
    'bootstrap': [True, False]
}
rf = RandomForestClassifier(random_state=42)
search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    verbose=1,
    n_jobs=-1
)
search.fit(X_train_ohe, y_train)
print('Best Parameters:', search.best_params_)


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20, 'bootstrap': False}


## 8. Model Evaluation
Evaluate the model's performance on training and test sets.


In [23]:
from sklearn.pipeline import make_pipeline
model_rf = make_pipeline(RandomForestClassifier(n_estimators=300,min_samples_split=4,min_samples_leaf=3,max_features='log2',max_depth=20,bootstrap=True,random_state=42))
model_rf.fit(X_train_ohe, y_train)
print('Train Accuracy:', model_rf.score(X_train_ohe, y_train))
print('Test Accuracy:', model_rf.score(X_test_ohe, y_test))


Train Accuracy: 0.79875
Test Accuracy: 0.7683333333333333


## 9. Results and Findings
- The best Random Forest model achieves a test accuracy of approximately 77-78%.
- Hyperparameter tuning further improves model performance.
- The notebook can be extended for further evaluation and experimentation.
